## ML Model for Dynamic Characters

### Imports

In [2]:
import keras
import numpy as np
import random

In [3]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


### Read Data

In [3]:
# get data 
# img = imageio.imread('../resources/textures/sphereTex.png')
# img = img[:, :, 0] # convert to grayscale
# rows = len(img) # rows
# cols = len(img[0]) # cols

# read data
# data = []
# with open("out.txt", "r") as text_file:
#     line = text_file.read()
#     data = line.split('\n')[:-1]
#     data = [ list(map(float, x.split(','))) for x in data ]
# data = np.array(data)

# # split data
# np.random.shuffle(data)
# partition = int(0.8 * len(data))
# X_train = data[:partition, :-1]
# y_train = data[:partition, -1] / 255.0
# X_test = data[partition:, :-1] 
# y_test = data[partition:, -1] / 255.0

In [97]:
# read data
name = "warrior"
train_path = "../resources/occlusion/" + name + "_train_data.txt"
test_path = "../resources/occlusion/" + name + "_test_data.txt"

data = []
bones = 0
vertices = 0
with open(train_path, "r") as text_file:
    raw = text_file.read()
    data = raw.split('\n')[:-1] # ignore empty list
    data = [ list(map(float, x.split(' '))) for x in data ]
    bones, vertices = data[0]
train_data = np.array(data[1:])

with open(test_path, "r") as text_file:
    raw = text_file.read()
    data = raw.split('\n')[:-1]
    data = [ list(map(float, x.split(' '))) for x in data ]
    bones, vertices = data[0]
test_data = np.array(data[1:])

# split data
x_len = int(3 * bones) # 3 for each euler angle

np.random.shuffle(train_data)
assert(len(train_data[0]) == x_len + vertices)
X_train = train_data[:, :x_len]
X_train = X_train / np.pi  #/ (2 * np.pi) # standardize
y_train = train_data[:, x_len:]

assert(len(test_data[0]) == x_len + vertices)
np.random.shuffle(test_data)
X_test = test_data[:, :x_len]
X_test = X_test / np.pi #/ (2 * np.pi) # standardize
y_test = test_data[:, x_len:]

### Build Model

<b>Inputs:</b> u, v, thetas
<br/><b>Output</b>: occlusion factor

In [104]:
# build model
model = keras.Sequential([
    keras.layers.Dense(2*vertices, input_shape=(x_len,), activation='relu'), # hidden
    keras.layers.Dense(vertices, activation='relu'), # hidden
    # keras.layers.Dense(512, activation='relu'), # hidden
    keras.layers.Dense(vertices) # output
])
# model = keras.Sequential([
#     keras.layers.Conv1D(512, 15, 3, input_shape=(x_len,1,)), # hidden
#     keras.layers.Conv1D(256, 12, 3, ), # hidden
#     keras.layers.Conv1D(128, 6, 3, ), # hidden
#     keras.layers.Flatten(),
#     keras.layers.Dense(1024, activation="relu"), # output
#     keras.layers.Dropout(0.5),
#     keras.layers.Dense(vertices)
# ])
model.compile(optimizer="rmsprop", loss=keras.losses.MeanSquaredError(), metrics="mse")

In [105]:
model.summary()

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_80 (Dense)            (None, 128)               31232     
                                                                 
 dense_81 (Dense)            (None, 128)               16512     
                                                                 
 dense_82 (Dense)            (None, 128)               16512     
                                                                 
 dense_83 (Dense)            (None, 128)               16512     
                                                                 
 dense_84 (Dense)            (None, 128)               16512     
                                                                 
 dense_85 (Dense)            (None, 128)               16512     
                                                                 
 dense_86 (Dense)            (None, 7568)            

### Set up Tensorboard

### Train Model

In [106]:
# train model
history = model.fit(X_train, y_train, epochs=100, batch_size=32)

Epoch 1/100
39/39 [==============================] - 1s 20ms/step - loss: 0.0137 - mse: 0.0137
Epoch 2/100
39/39 [==============================] - 1s 14ms/step - loss: 0.0096 - mse: 0.0096
Epoch 3/100
39/39 [==============================] - 0s 12ms/step - loss: 0.0091 - mse: 0.0091
Epoch 4/100
39/39 [==============================] - 0s 10ms/step - loss: 0.0087 - mse: 0.0087
Epoch 5/100
39/39 [==============================] - 0s 10ms/step - loss: 0.0084 - mse: 0.0084
Epoch 6/100
39/39 [==============================] - 0s 10ms/step - loss: 0.0082 - mse: 0.0082
Epoch 7/100
39/39 [==============================] - 0s 11ms/step - loss: 0.0080 - mse: 0.0080
Epoch 8/100
39/39 [==============================] - 1s 13ms/step - loss: 0.0078 - mse: 0.0078
Epoch 9/100
39/39 [==============================] - 0s 10ms/step - loss: 0.0076 - mse: 0.0076
Epoch 10/100
39/39 [==============================] - 0s 10ms/step - loss: 0.0074 - mse: 0.0074
Epoch 11/100
39/39 [=============================

### Test Accuracy

In [107]:
results = model.evaluate(X_test, y_test, batch_size=16)
print("test loss, test acc:", results)

12/12 [==============================] - 0s 5ms/step - loss: 0.0112 - mse: 0.0112
test loss, test acc: [0.01118133869022131, 0.01118133869022131]


**Conv:**
train ~ 0.004, test ~ 0.010 (mse)<br/>
**Dense:**
train ~ 0.0023, test ~ 0.011 mse (mse)

In [7]:
# # generate texture using model
# dim = 512
# inputs = []
# for r in range(dim,0,-1):
#     for c in range(dim):
#         x = [c/(dim-1), r/(dim-1)]
#         inputs.append(x)
# outputs = model.predict(inputs)
# outputs = [ 255 if y[0] > 1 else 255*y[0] for y in outputs ]

8192/8192 [==============================] - 3s 375us/step


In [8]:
# # save image
# img = Image.new('L', (dim,dim))
# img.putdata(outputs)
# img.save('nnTexture.png')

In [40]:
model.predict([
    [0.94197,0.61064,0],
    [0.85878,0.47439,0],
    [0.86127,0.47565,0],
    [0.84563,0.45853,0],
    [0.84873,0.46279,0],
    [0.92041,0.6823,0],
    [0.91853,0.68184,0]
])

1/1 [==============================] - 0s 161ms/step


In [38]:
wab = model.get_weights()

In [39]:
# manual evaluation 2 hidden layers: input = [u, v, theta]
def manual_eval(input):
    w1 = wab[0]
    b1 = wab[1]
    w2 = wab[2]
    b2 = wab[3]
    w3 = wab[4]
    b3 = wab[5]

    z1 = keras.activations.tanh(np.matmul(np.array(input), w1) + b1)
    z2 = keras.activations.tanh(np.matmul(z1, w2) + b2)
    z3 = keras.activations.tanh(np.matmul(z2, w3) + b3)
    return z3.numpy()[0]

0.019228264243694542

### Save to output file

In [35]:
# save weights and biases as buffer
output_buf = str(len(model.layers)) + "\n" # layer count

for i in range(0, len(wab)-1, 2): # iterate through layers
    weights = wab[i]
    output_buf += str(weights.shape[0]) + " " + str(weights.shape[1]) + "\n" # input size output size
    for neuron in weights: # iterate through
        for edges in neuron:
            output_buf += str(edges) + " "
    for biases in wab[i+1]:
        output_buf += str(biases) + " "
    output_buf = output_buf[:-1] + "\n" # ignore extraneous space and end line
    

In [36]:
with open("../resources/evaluators/model.txt", "w") as text_file:
    text_file.write(output_buf) # add more digits

# Second Model

### Create Model

Inputs: Joint Angles
Outputs: AO values for each vertex

In [1]:
import keras
import numpy as np

In [44]:
# read data
header = []
data = []
with open("../resources/occlusion/dataAbs.txt", "r") as text_file:
    line = text_file.read()
    data = [ list(map(float, x.split(' '))) for x in line.split('\n')[:-1] ]
    header = data[:1][0]
    data = data[1:]
data = np.array(data)

num_joints = int(header[0])
vertex_count = int(header[1])

# split data
np.random.shuffle(data)
partition = int(0.8 * len(data))
X_train = data[:partition, :num_joints]
y_train = data[:partition, num_joints:]
X_test = data[partition:, :num_joints]
y_test = data[partition:, num_joints:]

In [45]:
print(header)

[1.0, 1059.0]


In [79]:
# build model
model = keras.Sequential([
    keras.layers.Dense(1059, input_shape=(num_joints,), activation='tanh'), # hidden
    # keras.layers.Dense(?, activation='tanh'), # hidden
    keras.layers.Dense(vertex_count) # output
])
model.compile(optimizer="rmsprop", loss=keras.losses.MeanSquaredError(), metrics="mse")

In [80]:
# train model
history = model.fit(X_train, y_train, epochs=35, batch_size=64)

Epoch 1/35
2/2 [==============================] - 0s 15ms/step - loss: 0.0286 - mse: 0.0286
Epoch 2/35
2/2 [==============================] - 0s 14ms/step - loss: 0.0146 - mse: 0.0146
Epoch 3/35
2/2 [==============================] - 0s 20ms/step - loss: 0.0084 - mse: 0.0084
Epoch 4/35
2/2 [==============================] - 0s 21ms/step - loss: 0.0065 - mse: 0.0065
Epoch 5/35
2/2 [==============================] - 0s 14ms/step - loss: 0.0057 - mse: 0.0057
Epoch 6/35
2/2 [==============================] - 0s 24ms/step - loss: 0.0057 - mse: 0.0057
Epoch 7/35
2/2 [==============================] - 0s 22ms/step - loss: 0.0057 - mse: 0.0057
Epoch 8/35
2/2 [==============================] - 0s 14ms/step - loss: 0.0056 - mse: 0.0056
Epoch 9/35
2/2 [==============================] - 0s 22ms/step - loss: 0.0050 - mse: 0.0050
Epoch 10/35
2/2 [==============================] - 0s 18ms/step - loss: 0.0050 - mse: 0.0050
Epoch 11/35
2/2 [==============================] - 0s 11ms/step - loss: 0.0036 

In [81]:
wab = model.get_weights()

In [82]:
# save weights and biases as buffer
output_buf = str(len(model.layers)) + "\n" # layer count

for i in range(0, len(wab)-1, 2): # iterate through layers
    weights = wab[i]
    output_buf += str(weights.shape[0]) + " " + str(weights.shape[1]) + "\n" # input size output size
    for neuron in weights: # iterate through
        for edges in neuron:
            output_buf += str(edges) + " "
    for biases in wab[i+1]:
        output_buf += str(biases) + " "
    output_buf = output_buf[:-1] + "\n" # ignore extraneous space and end line

In [83]:
with open("../resources/evaluators/modelAbs.txt", "w") as text_file:
    text_file.write(output_buf) # add more digits

In [90]:
p = model.predict([
    X_test[0]
])


1/1 [==============================] - 0s 53ms/step


array([[ 0.00416847, -0.00844724,  0.00868796, ...,  0.00885363,
        -0.00669841,  0.00817197]])